In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# OpenAIEmbeddings

1. Embedding 개요
    - 텍스트 데이터를 벡터 형태의 숫자로 변환하여 기계 학습 모델이 처리할 수 있도록 하는 과정 
    - 문서 검색, 텍스트 유사도 계산, 추천 시스템 등 다양한 작업에 활용 

2. 주요 특징 비교 

| 특징 | 일반적인 임베딩 | OpenAIEmbeddings |
|-----|-------------|------------------|
| 기술 기반 | Word2Vec, GloVe, FastText 등 | Transformer 계열 (eg, `text-embedding_ada-002`) |
| 문맥 이해 | 단어 중심 | 문맥 기반의 동적인 임베딩 생성 |
| 언어 지원 | 영어 (추가 학습 필요) | 다국어 지원 |
| 커스터마이징 | 직접 학습 가능 | 불가능 |


3. 지원 모델 

| MODEL                  | PAGES PER DOLLAR | PERFORMANCE ON MTEB EVAL | MAX INPUT |
|------------------------|------------------|---------------------------|-----------|
| text-embedding-3-small | 62,500           | 62.3%                     | 8191      |
| text-embedding-3-large | 9,615            | 64.6%                     | 8191      |
| text-embedding-ada-002 | 12,500           | 61.0%                     | 8191      |

[Reference] https://python.langchain.com/docs/integrations/text_embedding/openai/

In [3]:
from langchain_openai import OpenAIEmbeddings

In [4]:
# default model
OpenAIEmbeddings().model

'text-embedding-ada-002'

In [5]:
# 임베딩 정의
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

## 쿼리 임베딩 

- `embeddings.embed_query()` 
- 텍스트를 벡터 공간에 매핑하여 의미적으로 유사한 텍스트를 찾거나 텍스트 간의 유사도를 계산하는 데 사용

In [6]:
# 샘플 데이터
sample_text = "임베딩 결과를 확인하기 위한 샘플 데이터 입니다. "

In [7]:
# 텍스트 임베딩 후 쿼리 결과 생성
query_result = embeddings.embed_query(sample_text)

In [8]:
print(len(query_result))
print(query_result[:10])

1536
[0.015623251907527447, 0.057504069060087204, 0.018380295485258102, -0.01630157232284546, 0.03365344554185867, -0.012362937442958355, -0.01200189534574747, -0.02201259322464466, 0.016979891806840897, -0.07255841046571732]


## 문서 임베딩 

- `embeddings.embed_documents()` 

In [9]:
docs_result = embeddings.embed_documents([sample_text, sample_text, sample_text])

In [10]:
print(len(docs_result))
print(len(docs_result[0]))

3
1536


In [11]:
docs_result[0]

[0.015623251907527447,
 0.057504069060087204,
 0.018380295485258102,
 -0.01630157232284546,
 0.03365344554185867,
 -0.012362937442958355,
 -0.01200189534574747,
 -0.02201259322464466,
 0.016979891806840897,
 -0.07255841046571732,
 0.03273443132638931,
 0.05216503143310547,
 -0.004113685339689255,
 -0.006701149512082338,
 -0.02028396911919117,
 -0.041486956179142,
 -0.02579805813729763,
 -0.001172017422504723,
 0.000823967915493995,
 0.029561642557382584,
 0.016367215663194656,
 -0.019649412482976913,
 0.008101552724838257,
 0.003908548038452864,
 -0.005038170609623194,
 -0.020721595734357834,
 0.005601614248007536,
 -0.02870827168226242,
 0.015721717849373817,
 -0.00840242113918066,
 -0.03472563251852989,
 -0.01971505582332611,
 0.027089055627584457,
 -0.025163501501083374,
 0.049626801162958145,
 -0.018577227368950844,
 -0.024353891611099243,
 -0.01810677908360958,
 0.006039240397512913,
 -0.020831001922488213,
 0.031399670988321304,
 0.005153047386556864,
 0.0023768567480146885,
 0.0

## 차원 지정 

- `text-embedding-3` 모델을 사용하면 반환되는 임베딩의 크기를 지정할 수 있음
- 기본적으로 `text-embedding-3-small`는 1536 차원의 임베딩 반환
- 벡터DB 차원에 맞게 조정하여 사용

In [12]:
embeddings.model

'text-embedding-3-small'

In [13]:
len(docs_result[0])

1536

In [14]:
# 차원 조정 (1024)
embeddings_ = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

len(embeddings_.embed_documents([sample_text])[0])

1024

## 유사도 계산

In [15]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# 코사인 유사도 계산 함수
# 계산하면 2차원 배열로 반환되어, 계산값(스칼라)만 가져오도록 반영 
def calculate_similarity_sentence(a, b):
    return cosine_similarity([a], [b])[0][0]

In [17]:
# 예제 문장들
sentence1 = "AI 기술은 현대 사회에서 점점 더 중요한 역할을 맡고 있습니다."
sentence2 = "인공지능 기술은 오늘날 다양한 산업 분야에서 중요한 역할을 하고 있습니다."
sentence3 = "안녕하세요? 김영희 입니다. 만나서 반갑습니다. "
sentence4 = "Artificial intelligence is becoming increasingly vital in modern society."
sentence5 = "The weather today is sunny and perfect for a picnic."
sentence6 = "I love the vanilla ice cream !!!"

In [18]:
# 임베딩 정의
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [19]:
# 각 문장의 임베딩 벡터 계산
sentences = [sentence1, sentence2, sentence3, sentence4, sentence5, sentence6]
vectors = [embeddings.embed_query(sentence) for sentence in sentences]

In [20]:
# 결과 확인
for i, sentence in enumerate(vectors):
    for j, other_sentence in enumerate(vectors):
        if i < j:
            print(
                f"[유사도 {calculate_similarity_sentence(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

[유사도 0.9211] AI 기술은 현대 사회에서 점점 더 중요한 역할을 맡고 있습니다. 	 <=====> 	 인공지능 기술은 오늘날 다양한 산업 분야에서 중요한 역할을 하고 있습니다.
[유사도 0.7603] AI 기술은 현대 사회에서 점점 더 중요한 역할을 맡고 있습니다. 	 <=====> 	 안녕하세요? 김영희 입니다. 만나서 반갑습니다. 
[유사도 0.8521] AI 기술은 현대 사회에서 점점 더 중요한 역할을 맡고 있습니다. 	 <=====> 	 Artificial intelligence is becoming increasingly vital in modern society.
[유사도 0.6928] AI 기술은 현대 사회에서 점점 더 중요한 역할을 맡고 있습니다. 	 <=====> 	 The weather today is sunny and perfect for a picnic.
[유사도 0.6805] AI 기술은 현대 사회에서 점점 더 중요한 역할을 맡고 있습니다. 	 <=====> 	 I love the vanilla ice cream !!!
[유사도 0.7584] 인공지능 기술은 오늘날 다양한 산업 분야에서 중요한 역할을 하고 있습니다. 	 <=====> 	 안녕하세요? 김영희 입니다. 만나서 반갑습니다. 
[유사도 0.8707] 인공지능 기술은 오늘날 다양한 산업 분야에서 중요한 역할을 하고 있습니다. 	 <=====> 	 Artificial intelligence is becoming increasingly vital in modern society.
[유사도 0.7076] 인공지능 기술은 오늘날 다양한 산업 분야에서 중요한 역할을 하고 있습니다. 	 <=====> 	 The weather today is sunny and perfect for a picnic.
[유사도 0.6838] 인공지능 기술은 오늘날 다양한 산업 분야에서 중요한 역할을 하고 있습니다. 	 <=====> 	 I love the vanilla ice cream !!!
[유사도

-----
** End of Documents **